In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/credit-debit dataset.csv
/kaggle/input/dataset/TransactionDataset1.csv


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



In [7]:

data = pd.read_csv('/kaggle/input/dataset/TransactionDataset1.csv')


selected_features = [
    'kyc_status', 'days_since_kyc_incomplete', 'transaction_amount',
    'home_branch', 'transaction_location', 'transaction_method',
    'transaction_category', 'transaction_merchant', 'transaction_time',
    'average_expenditure', 'comparison_with_avg_expenditure',
    'transaction_count_7_days', 'fraud_indicator'
]

data = data[selected_features]


X = data.drop('fraud_indicator', axis=1)
y = data['fraud_indicator']


numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])


categorical_features = X.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
X[categorical_features] = X[categorical_features].apply(label_encoder.fit_transform)



pca = PCA(n_components=12)  
X_pca = pca.fit_transform(X)



In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train Isolation Forest
isolation_forest_model = IsolationForest(contamination=0.02)
isolation_forest_model.fit(X_train)

# Predict anomalies using Isolation Forest
isolation_forest_predictions = isolation_forest_model.predict(X_test)

# Convert X_test to DataFrame
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Combine Isolation Forest results with original features
X_combined = pd.concat([X_test_df, pd.DataFrame(isolation_forest_predictions, columns=['isolation_forest'])], axis=1)

# Train One-Class SVM on the combined features
svm_model = OneClassSVM(nu=0.4, kernel='rbf') 
svm_model.fit(X_combined)

# Make predictions using One-Class SVM
svm_predictions = svm_model.predict(X_combined)

# Evaluate the combined model
print(confusion_matrix(y_test, svm_predictions))
print(classification_report(y_test, svm_predictions))


[[   0    0    0]
 [ 225    0 2337]
 [ 176    0 1262]]
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00      2562
           1       0.35      0.88      0.50      1438

    accuracy                           0.32      4000
   macro avg       0.12      0.29      0.17      4000
weighted avg       0.13      0.32      0.18      4000



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.